![Parked car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries, insurance is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance has requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to use simple Logistic Regression, identifying the single feature that results in the best-performing model, as measured by accuracy.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table (below) detailing the column names and descriptions below.

## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [20]:
# Import required libraries
library(readr)
library(dplyr)
library(glue)
library(yardstick)

# Start coding!
# Read in the data
car_insurance <- read_csv("car_insurance.csv")

# Explore the data
str(car_insurance)
colSums(is.na(car_insurance))
summary(car_insurance$credit_score)
summary(car_insurance$annual_mileage)

# Impute any missing values and remove the id column
car_insurance$credit_score[is.na(car_insurance$credit_score)] <- mean(car_insurance$credit_score, na.rm = TRUE)
car_insurance$annual_mileage[is.na(car_insurance$annual_mileage)] <- mean(car_insurance$annual_mileage, na.rm = TRUE)
feature_df <- data.frame(features = c(names(subset(car_insurance, select = -c(id, outcome)))))

# Create Logistic Regression models for each single feature and assess accuracy
for (feature in feature_df$features) {
	model <- glm(glue('outcome ~ {feature}'), data = car_insurance, family = "binomial")
	prediction_values <- round(fitted(model))
	accuracy <- length(which(prediction_values == car_insurance$outcome)) / length(car_insurance$outcome)
	feature_df[which(feature_df$feature == feature), "accuracy"] = accuracy
}

# Find the best feature with the best accuracy
best_feature <- feature_df$features[which.max(feature_df$accuracy)]
best_accuracy <- max(feature_df$accuracy)

# Create the output data frame called best_feature_df
best_feature_df <- data.frame(best_feature, best_accuracy)
best_feature_df

Rows: 10000 Columns: 19
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (19): id, age, gender, race, driving_experience, education, income, cred...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


spc_tbl_ [10,000 × 19] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ id                 : num [1:10000] 569520 750365 199901 478866 731664 ...
 $ age                : num [1:10000] 3 0 0 0 1 2 3 1 2 2 ...
 $ gender             : num [1:10000] 0 1 0 1 1 0 1 0 0 0 ...
 $ race               : num [1:10000] 1 1 1 1 1 1 1 1 1 1 ...
 $ driving_experience : num [1:10000] 0 0 0 0 1 2 3 0 2 0 ...
 $ education          : num [1:10000] 2 0 2 3 0 2 2 3 3 2 ...
 $ income             : num [1:10000] 3 0 1 1 1 3 3 1 1 3 ...
 $ credit_score       : num [1:10000] 0.629 0.358 0.493 0.206 0.388 ...
 $ vehicle_ownership  : num [1:10000] 1 0 1 1 1 1 0 0 0 1 ...
 $ vehicle_year       : num [1:10000] 1 0 0 0 0 1 1 1 0 0 ...
 $ married            : num [1:10000] 0 0 0 0 0 0 1 0 1 0 ...
 $ children           : num [1:10000] 1 0 0 1 0 1 1 1 0 1 ...
 $ postal_code        : num [1:10000] 10238 10238 10238 32765 32765 ...
 $ annual_mileage     : num [1:10000] 12000 16000 11000 11000 12000 13000 13000 14000 13000 1100

id                 age              gender                race 
                  0                   0                   0                   0 
 driving_experience           education              income        credit_score 
                  0                   0                   0                 982 
  vehicle_ownership        vehicle_year             married            children 
                  0                   0                   0                   0 
        postal_code      annual_mileage        vehicle_type speeding_violations 
                  0                 957                   0                   0 
               duis      past_accidents             outcome 
                  0                   0                   0

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
 0.0534  0.4172  0.5250  0.5158  0.6183  0.9608     982 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   2000   10000   12000   11697   14000   22000     957 

best_feature,best_accuracy
<chr>,<dbl>
driving_experience,0.7771
